This code is very basic, I am not a coder or good at math. Please feel free to improve it in any way.
Tyler Hall 2.24.24
Git hub repo [NNNavigator](https://github.com/angrysky56/NNNavigator)

In [33]:
# Use CLI to run: "conda create -n jupy python=3.12" then activate the kernel in VS Code notebooks.
# uncomment the line below to install the required packages, add more as required.
# %pip install numpy pyarrow scipy matplotlib 

In [34]:
import json
import numpy as np
from scipy.signal import convolve2d
import matplotlib.pyplot as plt
from PIL import Image
import io
import os 

# Preset configurations for quick access
presets = {
    "Fingerprint": {"size": 100, "Du": 0.16, "Dv": 0.08, "F": 0.035, "k": 0.060},
    "Maze": {"size": 100, "Du": 0.07, "Dv": 0.06, "F": 0.035, "k": 0.065},
    "Spots and Waves": {"size": 100, "Du": 0.19, "Dv": 0.05, "F": 0.060, "k": 0.062},
    "Moving Spots": {"size": 100, "Du": 0.12, "Dv": 0.08, "F": 0.020, "k": 0.055},
    "Chaos and Threads": {"size": 100, "Du": 0.10, "Dv": 0.10, "F": 0.030, "k": 0.057},
    "Zigzag Waves": {"size": 100, "Du": 0.16, "Dv": 0.08, "F": 0.062, "k": 0.061}
}

# List of available color maps
color_map = ["viridis", "plasma", "inferno", "magma", "cividis"]

# Define the kernel for the convolution that represents the Laplacian operator
laplacian_kernel = np.array([[0.05, 0.2, 0.05],
                             [0.2, -1, 0.2],
                             [0.05, 0.2, 0.05]])

def laplacian_convolution(Z):
    return convolve2d(Z, laplacian_kernel, mode='same', boundary='fill', fillvalue=0)

def simulate_turing_patterns(preset, color_map, size, Du, Dv, F, k, initial_condition, time_steps, view_mode):
    if preset:
      # Load preset parameters
      size = presets[preset]['size']
      Du = presets[preset]['Du']
      Dv = presets[preset]['Dv']
      F = presets[preset]['F']
      k = presets[preset]['k']
    
    # Initialize concentration of u and v
    U = np.ones((size, size))
    V = np.zeros((size, size))
    
    # Add initial conditions for V
    middle = size // 2
    if initial_condition == "small square":
        U[middle-5:middle+5, middle-5:middle+5] = 0.50
        V[middle-5:middle+5, middle-5:middle+5] = 0.25
    elif initial_condition == "large square":
        U[middle-10:middle+10, middle-10:middle+10] = 0.50
        V[middle-10:middle+10, middle-10:middle+10] = 0.25
    elif initial_condition == "random":
        np.random.seed(0)
        V = np.random.rand(size, size)
 
    # Time-stepping loop
    for t in range(time_steps):
        # Apply the Laplacian function using convolution
        Lu = laplacian_convolution(U)
        Lv = laplacian_convolution(V)
        
        # Reaction-diffusion equations, updating only the interior
        uvv = U[1:-1, 1:-1] * V[1:-1, 1:-1] ** 2
        U[1:-1, 1:-1] += Du * Lu[1:-1, 1:-1] - uvv + F * (1 - U[1:-1, 1:-1])
        V[1:-1, 1:-1] += Dv * Lv[1:-1, 1:-1] + uvv - (F + k) * V[1:-1, 1:-1]

         # Visualization
    # Save the plot to a BytesIO object and convert to an image
    buf = io.BytesIO()
    plt.imshow(U, cmap=color_map)
    plt.axis('off')
    plt.tight_layout()
    plt.savefig(buf, format='png')
    buf.seek(0)
    img = Image.open(buf)
    plt.close()
    return img



 **The feed rate (F) typically controls how quickly the activator is fed into the system, and the kill rate (k) controls how quickly the inhibitor removes the activator. Higher diffusion rates of the activator (Du) relative to the inhibitor (Dv) can lead to spot patterns, while the opposite can lead to stripe patterns.**

Based on general knowledge, here are some examples that could illustrate the influence of the parameters on Turing patterns:

1. **Feed Rate (F)**:
   - **Low Feed Rate**: A low feed rate (e.g., F = 0.01) means the activator is supplied slowly to the system. This might result in patterns that take longer to emerge or are less pronounced.
   - **High Feed Rate**: A high feed rate (e.g., F = 0.12) supplies the activator more quickly, potentially leading to more rapid pattern formation and more vibrant or chaotic patterns.

2. **Kill Rate (k)**:
   - **Low Kill Rate**: A low kill rate (e.g., k = 0.02) implies the inhibitor removes the activator slowly, which can allow the activator to accumulate and may lead to the formation of spot-like patterns.
   - **High Kill Rate**: A high kill rate (e.g., k = 0.07) can quickly remove the activator, potentially leading to the formation of stripe-like patterns as the activator is not able to accumulate to the same extent.

3. **Diffusion Rates (Du, Dv)**:
   - **Higher Diffusion Rate of Activator (Du > Dv)**: If the activator diffuses more rapidly than the inhibitor (e.g., Du = 0.16 and Dv = 0.08), it can lead to the formation of spots or blobs as the activator spreads out quickly from its source.
   - **Higher Diffusion Rate of Inhibitor (Du < Dv)**: If the inhibitor diffuses more rapidly (e.g., Du = 0.08 and Dv = 0.16), it tends to suppress the activator more uniformly, which can lead to stripe-like or labyrinthine patterns.

The exact behavior can vary significantly based on the specific model and the interaction of these parameters. Experimentation and simulation are often used to explore the vast parameter space of Turing patterns. For educational or illustrative purposes, these examples can serve as a starting point to understand the influence of each parameter. If you need more detailed and specific examples, consulting academic literature on reaction-diffusion systems and Turing patterns would be the best approach.

In [35]:

import gradio as gr

# We need to add the new 'view_mode' parameter to the Gradio interface
iface = gr.Interface(
    simulate_turing_patterns,
    inputs=[
        gr.Dropdown(list(presets.keys()) + [None], label="Parameter Presets", value=None),
        gr.Dropdown(color_map, label="Color Map", value="viridis"),
        gr.Slider(10, 1000, step=10, value=100, label="Grid Size: Higher values zoom out, can take a while."),
        gr.Slider(minimum=0.001, maximum=1, step=0.001, value=0.2, label="Diffusion Rate of Activator: U (Du)"),
        gr.Slider(minimum=0.001, maximum=1, step=0.001, value=0.03, label="Diffusion Rate of Inhibitor: V (Dv)"),
        gr.Slider(minimum=0.001, maximum=1, step=0.001, value=0.03, label="Feed Rate: Speed of Activator supply (F)"),
        gr.Slider(minimum=0.001, maximum=1, step=0.001, value=0.05, label="Kill Rate: Speed of Activator removal (k)"),
        gr.Radio(choices=["small square", "large square", "random"], value= "large square", label="Initial Condition"),
        gr.Slider(minimum=100, maximum=15000, step=100, value=2000, label="Total Number of Iterations"),
        gr.Radio(choices=['U', 'V', 'U+V'], label="View Mode", value='U')
    ],
    outputs="image",
    live=False
)

iface.launch()


Running on local URL:  http://127.0.0.1:7875

To create a public link, set `share=True` in `launch()`.
